# ⭐**Dynamic Chat Agents**


## Table of Contents

- [***1. Introduction to Graph-Based Chatbots***](#1-introduction-to-graph-based-chatbots)
- [***2. Building an Agent with LangGraph***](#2-building-an-agent-with-langgraph)
- [***3. Streaming Responses***](#3-streaming-responses)
- [***4. Adding External Tools***](#4-adding-external-tools)
- [***5. Adding Memory***](#5-adding-memory)


In [ ]:
# Setup cell: Importing all necessary modules

from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image, display
from langgraph.checkpoint.memory import MemorySaver



## 1. Introduction to Graph-Based Chatbots

LangGraph enables the construction of agentic chatbots by defining workflows as stateful graphs.

### Key Concepts

- **Graph State**: Controls tool usage and the order of execution.
- **Agent State**: Tracks the task completion and maintains conversation history.
- **Nodes**: Functions or tool calls.
- **Edges**: Rules that direct the graph from one node to another.
- Pre-built nodes include START and END.

- 📌 *Think of a graph as a map of decisions or steps a chatbot takes when responding to user input.*




## 2. Building an Agent with LangGraph

We define a language model, a state type, and initialize the LangGraph system.

### Define the LLM and State


In [ ]:
# This cell initializes the language model
llm = ChatOpenAI(model="gpt-4o-mini", api_key="OPENAI_API_KEY")

In [ ]:
# This cell defines the State structure for the graph
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
# This cell initializes the graph builder
graph_builder = StateGraph(State)

In [ ]:
# Define the chatbot response function
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

# Add chatbot node to the graph
graph_builder.add_node("chatbot", chatbot)

In [ ]:
# Define edges between START -> chatbot -> END
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

# Compile the graph
graph = graph_builder.compile()


## 3. Streaming Responses

LangGraph supports streaming responses which allow for real-time feedback.

### Function to Stream Events


In [ ]:
# This function streams chatbot responses
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [("user", user_input)]}):
        for value in event.values():
            print("Agent:", value["messages"])

# Example query
user_query = "Who is Mary Shelley?"
stream_graph_updates(user_query)


## 4. Adding External Tools

LangGraph supports integration with tools such as Wikipedia.

### Add Wikipedia Tool


In [ ]:
# Set up Wikipedia API tool
api_wrapper = WikipediaAPIWrapper(top_k_results=1)
wikipedia_tool = WikipediaQueryRun(api_wrapper=api_wrapper)
tools = [wikipedia_tool]

# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)

# Update chatbot to use tool
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [ ]:
# Add chatbot and tool nodes
graph_builder.add_node("chatbot", chatbot)
tool_node = ToolNode(tools=[wikipedia_tool])
graph_builder.add_node("tools", tool_node)

# Add conditional edge and routing
graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)


## 5. Adding Memory

LangGraph can persist conversations using memory modules.

### Configure Memory


In [ ]:
# Initialize memory and compile the graph
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [ ]:
# Function for streaming with memory
def stream_memory_responses(user_input: str):  
    config = {"configurable": {"thread_id": "single_session_memory"}}
    for event in graph.stream({"messages": [("user", user_input)]}, config):
        for value in event.values():
            if "messages" in value and value["messages"]:
                print("Agent:", value["messages"])

# Streaming sample queries
stream_memory_responses("What is the Colosseum?")
stream_memory_responses("Who built it?")